<h1 align="center">LING530F: Deep Learning for NLP</h1>
<h1 align="center">Assignment 2 : Implicit Emotion Detection</h1>
<h1 align="center">Traditional Machine Learning Models</h1>
<h2 align="center"> Arun Rajendran(86611860)</h2>


In [1]:
%%capture
!pip install GoogleDriveDownloader
import nltk
nltk.download('popular')

In [2]:
# Download the data

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='11surQQr3jbmzHDKgEA5c46CKA23Nf7PA',                                 # Id of file to be downloaded
                                    dest_path='data/dev.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1TokPKns1uZRAW-GDN0fUz259haU2ZZAW',                                 # Id of file to be downloaded
                                    dest_path='data/test.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1oBiUaUHLrXehWF570xLyQNTjD9QU4fPY',                                 # Id of file to be downloaded
                                    dest_path='data/train.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1ewgcXWBioeMVJlKsZK9gLv3hWxGDqmiz',                                 # Id of file to be downloaded
                                    dest_path='data/trial-v3.labels')                                                    # Destination path

In [3]:
# Replace the fake labels in dev file with correct labels
labels = open('data/trial-v3.labels','r').readlines()
lines = open('data/dev.csv','r').readlines()
correct_lines=[]
for label, line in zip(labels, lines):
        line=line.rstrip('\n')
        text = line.split(',')
        text[0]=label
        correct_line=[''.join(text[0]),''.join(text[1:])]
        correct_lines.append(correct_line)

In [4]:
# Write the clean dev data in new file
out = open('data/clean_dev.csv',mode='w')                                      
for line in correct_lines:
    line[0]=line[0].rstrip('\n')
    out.write(line[0])
    out.write(',')
    out.write(line[1])
    out.write('\n')
out.close()                                                                    

In [5]:
# Read train and dev data
%%capture
import pandas as pd
X_train = pd.read_table('data/train.csv',sep=',',index_col=False, error_bad_lines=False)
X_test = pd.read_csv('data/clean_dev.csv',sep=',')

In [6]:
X_train.head(10)

,label,tweet
0,anger,@USERNAME A little [#TRIGGERWORD#] that I am n...
1,disgust,@USERNAME @USERNAME It's pretty [#TRIGGERWORD#...
2,fear,Apparently I've been black mailing my brother ...
3,fear,Republicans are so [#TRIGGERWORD#] that people...
4,sad,Katy once felt so [#TRIGGERWORD#] that she bar...
5,surprise,@USERNAME @USERNAME I'm ever more [#TRIGGERWOR...
6,disgust,@USERNAME @USERNAME Woohoo! I'm [#TRIGGERWORD#...
7,surprise,@USERNAME — pain I'm in from being so lonely. ...
8,disgust,pretty [#TRIGGERWORD#] that a white republican...
9,sad,@USERNAME no cause I don't wanna be that [#TRI...


In [7]:
X_test.head(10)

,label1,tweet
0,surprise,@USERNAME I'm just [#TRIGGERWORD#] that Sears ...
1,disgust,Adjoa looks [#TRIGGERWORD#] that Nick has put ...
2,fear,A key component of why it's so much fun to rea...
3,disgust,I am [#TRIGGERWORD#] that these women are bein...
4,surprise,@USERNAME really?! That's actually [#TRIGGERWO...
5,sad,It's very [#TRIGGERWORD#] that Presidents even...
6,disgust,@USERNAME not all but a lot of humans are [#TR...
7,sad,@USERNAME how does it make you [#TRIGGERWORD#]...
8,disgust,@USERNAME @USERNAME [#TRIGGERWORD#] when elsew...
9,joy,make you feel like you're not good enough dump...


In [8]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from scipy.stats import itemfreq
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,HashingVectorizer
from sklearn.metrics import confusion_matrix

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()  
  
pd.options.mode.chained_assignment = None

In [9]:
X_train.label.value_counts()

joy         27672
disgust     25351
surprise    25183
anger       25180
fear        24991
sad         22980
Name: label, dtype: int64

In [10]:
def lemmatize_sentences(sentence):
    tokens = sentence.split()
    lm_tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lm_tokens)

In [11]:
def clean_text(content):
  content=content.str.lower()                                                         # Convert to Lowercase
  content=content.str.replace('#triggerword#','')                                     # Remove triggerword tags
  content=content.str.replace('username','')                                          # Remove username tags
  content=content.str.replace('http\S+|www.\S+', '')                                  # Remove Links
  content=content.str.replace('\s+', ' ')                                             # Remove multiple spaces
  content=content.str.replace('[^A-Za-z\s]+', '')                                     # Remove irrelevant characters other than alphabets and space
  #content=content.apply(lemmatize_sentences)
  return content
  # Stemming
  # Remove sparse terms

In [12]:
X_train['tweet'] = clean_text(X_train['tweet'])
X_test['tweet']=clean_text(X_test['tweet'])

In [13]:
X_train.tweet[0:9]

0     a little  that i am not invited for drinks an...
1      its pretty  that there would even be stock p...
2    apparently ive been black mailing my brother s...
3    republicans are so  that people may treat peop...
4    katy once felt so  that she barely had the str...
5      im ever more  that potus  flotus take it in ...
6      woohoo im  because i breastfed my babies in ...
7      pain im in from being so lonely but im  that...
8    pretty  that a white republican man keeps intr...
Name: tweet, dtype: object

In [14]:
y_train=X_train.label
X_train = X_train.drop(['label'],axis=1)
y_test=X_test.label1
X_test = X_test.drop(['label1'],axis=1)

# Label encode the target variable of train & dev data
le1=LabelEncoder()
y_train=le1.fit_transform(y_train)

le2=LabelEncoder()
y_test=le2.fit_transform(y_test)

In [15]:
# Extracting features from text files
count_vect = CountVectorizer(ngram_range=(1,4))
X_train_counts = count_vect.fit_transform(X_train.tweet)
X_test_counts =count_vect.transform(X_test.tweet)
print('Shape of Term Frequency Matrix: ',X_train_counts.shape)

Shape of Term Frequency Matrix:  (151357, 3747354)


In [16]:
# Machine Learning
# Training Naive Bayes (NB) classifier + BOW model
clf = MultinomialNB().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
from sklearn.metrics import f1_score
f1_score(y_test, predicted, average='macro')

0.5852394508367251

In [17]:
# Training Linear SVM classifier + BOW model

from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
from sklearn.metrics import f1_score
f1_score(y_test, predicted, average='macro')

0.5977939767338595

In [18]:
# Training Logisitic Regression(LogReg) classifier + BOW model 

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial').fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
from sklearn.metrics import f1_score
f1_score(y_test, predicted, average='macro')

0.5724634386982159

In [19]:
def print_acc(model):
    y_pred = model.predict(X_test.tweet)
    mirco_f1= f1_score(y_test, y_pred, average='macro')
    print(mirco_f1)

In [20]:
# Training SVM + BOW + TF-IDF model

svm_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))), ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])
svm_clf = svm_clf.fit(X_train.tweet,y_train)
print_acc(svm_clf)

0.6123916337902315


In [21]:
# Training NB + BOW + TF-IDF model

nb_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.tweet,y_train)
print_acc(nb_clf)

0.57275368417417


In [22]:
# Training LogReg + BOW + TF-IDF model

lg_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial'))])
lg_clf = lg_clf.fit(X_train.tweet,y_train)
print_acc(lg_clf)

0.582516756775786


In [23]:
# Training Ensemble model

from sklearn.ensemble import VotingClassifier
model1 = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
model2 = MultinomialNB()
model3 = LinearSVC()
# create the ensemble model
eclf1 = VotingClassifier(estimators=[('lr', model1), ('nb', model2), ('svm', model3)], voting='hard')
ensemble = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))), ('tfidf', TfidfTransformer()), ('clf', eclf1)])
ensemble = ensemble.fit(X_train.tweet, y_train)
print_acc(ensemble)

0.6046228524717893


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### References

NLTK: Stemming and Lemmatization<br\>
https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization

Apply Stemmer to a column<br\>
https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word

Ensemble: Voting Classifier<br\>
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

Sample Pipeline<br\>
http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

TorchText and LSTM<br\>
http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

TorchText and CNN<br\>
https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/08.CNN-for-Text-Classification.ipynb
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

Saving and loading Pytorch Model<br\>
https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

Self Attention Explained<br\>
https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#self-attention

Self Attention Code for LSTM<br\>
https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py

LSTM + CNN model 
https://discuss.pytorch.org/t/cnn-layer-in-the-top-of-lstm/7941/5
https://discuss.pytorch.org/t/solved-concatenate-time-distributed-cnn-with-lstm/15435/4